In [1]:
import requests
import pandas as pd
from datetime import datetime, timedelta
import gc

In [2]:

URL="https://data.enedis.fr/api/records/1.0/search/?dataset=conso-inf36-region&q=horodate%3A%5B2020-06-04T00%3A00%3A00Z+TO+2020-06-05T00%3A30%3A00Z%5D&rows=10000&facet=horodate&facet=region&facet=profil&facet=plage_de_puissance_souscrite&facet=jour_max_du_mois_0_1&facet=semaine_max_du_mois_0_1"
URL2="https://data.enedis.fr/api/records/1.0/search/?dataset=conso-inf36-region&q=&rows=10000&facet=horodate&facet=region&facet=profil&facet=plage_de_puissance_souscrite&facet=jour_max_du_mois_0_1&facet=semaine_max_du_mois_0_1&horodate=2020-06-03T20:30:00+00:00"
response=requests.get(url=URL)




In [3]:
def get_data(releve):
    
    field=releve["fields"]
    data = {
        'horodate': field['horodate'],
        'region': field['region'],
        'code_region': field['code_region'],
        "total_energie_soutiree_wh" : field["total_energie_soutiree_wh"],
        'plage_de_puissance_souscrite': field['plage_de_puissance_souscrite'],
        'nb_points_soutirage': field['nb_points_soutirage'],
        'jour_max_du_mois_0_1': field['jour_max_du_mois_0_1'],
        'semaine_max_du_mois_0_1': field['semaine_max_du_mois_0_1'],
        'profil': field['profil']

    }
    return data

In [4]:
# 30 min c'est trop petit

dtformat = '%Y-%m-%dT%H.3A%M.3A%SZ' # format de la date requis par l'api; remplacer ensuite les . par des %

def time_travel(before, min):
    '''
    Permet de modifier la date en voyageant de 'hour' minutes dans le futur 

    'now : str 
        date actuelle
    
    'hour : int
        heures à additionner
    '''

    before = datetime.strptime(before, dtformat)
    futur = before + timedelta(minutes=min)
    return futur.strftime(dtformat)

before="2021-06-30T00.3A00.3A00Z"
#2020-07-18T09.3A30.3A00Z 

end="2021-08-15T00.3A00.3A00Z" #pour l'instant la bdd s'arrête au 30 septembre
end=datetime.strptime(end, dtformat)

list_releve=[]
i=0

while True:

     # on voyage 15 heures après pour récupérer un maximum de relevés électriques sur chaque heure 
    after = time_travel(before, min=30)
    if datetime.strptime(after, dtformat) > end:
        #si on atteint le 30 septembre, on sort de la boucle
        break
    
        
    #print("after : {} ".format(after))
    before_api=str(before).replace(".","%")
    after_api=str(after).replace(".","%")
    
    URL="https://data.enedis.fr/api/records/1.0/search/?dataset=conso-inf36-region&q=horodate%3A%5B{}+TO+{}%5D&rows=10000&facet=horodate&facet=region&facet=profil&facet=plage_de_puissance_souscrite&facet=jour_max_du_mois_0_1&facet=semaine_max_du_mois_0_1".format(before_api,after_api)
    response = requests.get(url=URL)  # envoie de la requête
    
    i+=1
    list_releve_temp=[]
    # on ajoute les relevés à notre df
    for releve in response.json()['records']:
        if "total_energie_soutiree_wh" in releve["fields"]:
            row = get_data(releve) 
            list_releve_temp.append(row) 
    
    list_releve_temp = sorted(list_releve_temp, key=lambda d: d['horodate'])

    before=list_releve_temp[len(list_releve_temp)-1]["horodate"]
    before=before.replace(":",".3A")
    before=before.replace("+00.3A00","Z")

    list_releve.extend(list_releve_temp)
    
    del list_releve_temp
    gc.collect()


#df_enedis = pd.DataFrame(list_releve)
#df_enedis.drop_duplicates(inplace=True)
#df_enedis=df_enedis.sort_values("horodate")

#df_enedis.to_csv("Données/enedis_api.csv",index=False) # enregistrement des données

In [5]:
df_enedis = pd.DataFrame(list_releve)
df_enedis.drop_duplicates(inplace=True)
df_enedis=df_enedis.sort_values("horodate")
df_enedis.reset_index(inplace=True,drop=True)
df_enedis.to_csv("Données/enedis_api_2021_5.csv",index=False) # enregistrement des données

In [6]:
df_enedis

,horodate,region,code_region,total_energie_soutiree_wh,plage_de_puissance_souscrite,nb_points_soutirage,jour_max_du_mois_0_1,semaine_max_du_mois_0_1,profil
0,2021-01-01T00:00:00+00:00,Auvergne-Rhône-Alpes,84,1.904600e+07,P3: ]6-9] kVA,60415,0,0,PRO1 (+ PRO1WE)
1,2021-01-01T00:00:00+00:00,Auvergne-Rhône-Alpes,84,1.415221e+09,P0: Total <= 36 kVA,1579671,0,0,RES2 (+ RES5)
2,2021-01-01T00:00:00+00:00,Auvergne-Rhône-Alpes,84,5.386863e+08,P3: ]6-9] kVA,561645,0,0,RES2 (+ RES5)
3,2021-01-01T00:00:00+00:00,Auvergne-Rhône-Alpes,84,1.423012e+07,P9: ]30-36] kVA,4969,0,0,RES2 (+ RES5)
4,2021-01-01T00:00:00+00:00,Bourgogne-Franche-Comté,27,6.409028e+06,P4: ]9-12] kVA,14600,0,0,PRO1 (+ PRO1WE)
...,...,...,...,...,...,...,...,...,...
1325053,2021-01-31T16:30:00+00:00,Nouvelle Aquitaine,75,1.047327e+07,P0: Total <= 36 kVA,20203,0,0,RES2WE
1325054,2021-01-31T16:30:00+00:00,Occitanie,76,1.126969e+07,P7: ]18-24] kVA,10653,0,0,PRO1 (+ PRO1WE)
1325055,2021-01-31T16:30:00+00:00,Occitanie,76,7.440550e+06,P8: ]24-30] kVA,5829,0,0,PRO1 (+ PRO1WE)
1325056,2021-01-31T16:30:00+00:00,Bourgogne-Franche-Comté,27,5.471922e+06,P7: ]18-24] kVA,5273,0,0,PRO1 (+ PRO1WE)


In [91]:

for releve in response.json()['records']:
    if "total_energie_soutiree_wh" in releve["fields"]:
        row = get_data(releve)  
        df_enedis = df_enedis.append(row, ignore_index=True)

before=row["horodate"]
before=before.replace(":",".3A")
before=before.replace("+00.3A00","Z")

